In [ ]:
!pip install openai tiktoken chromadb langchain

In [2]:
!pip install pypdf

In [104]:
from pypdf import PdfReader, PdfWriter
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from transformers import pipeline

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from urllib import request
from urllib.request import urlopen
import io

In [ ]:
import os
import openai

#use your openai api key here
my_key = os.getenv(OPENAI_API_KEY)

In [125]:
def load_pdf(url, file_path):
  writer = PdfWriter()
  remoteFile = urlopen(url).read()
  memoryFile = io.BytesIO(remoteFile)
  pdfFile = PdfReader(memoryFile)
  text = ""
  for page in range(len(pdfFile.pages)):
    text = text + pdfFile.pages[page].extract_text()
  with open(file_path, "w") as f:
    f.write(text)

In [128]:
def scan_pdf(file_path, text_path):
  reader = PdfReader(file_path)
  text = ""
  for page in range(len(reader.pages)):
    text = text + reader.pages[page].extract_text()
  with open('initial_text.txt', "w") as f:
    f.write(text)


def get_summary(file_path, openai_api_key):
  # Reat text file
  loader = TextLoader(file_path)
  document = loader.load()
  # Define prompt
  prompt_template = """
  Write a concise summary of the following text delimited by triple backquotes. Return your response in Russian in bullet points which covers the key points of the text.
  ```{text}```
  BULLET POINT SUMMARY:
  """
  prompt = PromptTemplate.from_template(prompt_template)
  # Define LLM chain
  llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k", openai_api_key=openai_api_key)
  llm_chain = LLMChain(llm=llm, prompt=prompt)
  # Define StuffDocumentsChain
  stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")
  answer = str(stuff_chain.run(document))
  return answer


def load_model():
  return pipeline('translation_en_to_rus', model='Helsinki-NLP/opus-mt-en-ru')


def process_data(text, translator):
    text_splitted = text.split('\n')
    tr_concat = ""
    for i in text_splitted:
      response = translator(i)[0]['translation_text']
      tr_concat = tr_concat + response + "\n"
    return tr_concat

In [129]:
def sum_arxiv(arxiv_paper_url, text_file_name, openai_api_key):
  # Read article from pdf
  load_pdf(arxiv_paper_url, text_file_name)
  # Summarize text with OpenAI
  summary = get_summary(text_file_name, openai_api_key)
  print(f"Summary:\n{summary}")
  return response

In [130]:
rus_summary = sum_arxiv('https://arxiv.org/pdf/1903.04936.pdf','initial_text_2.txt', my_key)

Summary:
- В данной статье представлена структура данных k-d tree и доказано, что вычисление соседей в этой структуре выполняется за ожидаемое логарифмическое время.
- k-d tree был представлен в 1977 году и имеет применение в поиске соседей и ортогональных диапазонах.
- Построение k-d tree занимает Θ(n·log(n)) времени и требует Θ(n) памяти.
- Ожидаемое время выполнения запроса на поиск соседей в k-d tree составляет O(log(n)).
- Доказательство основано на использовании бета-распределения и показывает практическую значимость k-d tree для вычисления соседей в геометрической обработке.
